In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from FetusGrowth_BWPrediction import *
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')

sklearn.exceptions.ConvergenceWarning('ignore')

In [3]:
data_df = pd.read_csv('seethapathyCleanedData.csv')
data = getMeasurements_int(data_df)
data = data.drop([i for i in range(len(data['GA'])) if list(data['GA'])[i][1] > 24]).reset_index(drop = True)
data = data.drop([i for i in range(len(data['GA'])) if list(data['GA'])[i][-1] < 27]).reset_index(drop = True)
ml_data = rmMultipleUS(data)
column_toDrop = ['NEONATAL_COMPLICATIONS']

X = ml_data.drop(column_toDrop, axis=1)
y = ml_data['BIRTH_WEIGHT']

# Birth weight prediction using data upto 24 and 35 Week Prediction

In [ ]:
''' Leave one out for birth weight prediction '''

loo = LeaveOneOut()
y_true_list, y_pred_list,fi_coeff_list = [],[],[]
df = pd.DataFrame(columns = ['Alpha', 'Score','BWPredicted24','BWPredicted35','BWActual','Parameter_coeff','intercept'])
pred_df = pd.DataFrame(columns = ['HC24','AC24','BPD24','FL24','A_HC24','A_AC24','A_BPD24','A_FL24',
                       'HC35','AC35','BPD35','FL35','A_HC35','A_AC35','A_BPD35','A_FL35',
                       'HC24_atDel','AC24_atDel','BPD24_atDel','FL24_atDel','A_HC24_atDel',
                       'A_AC24_atDel','A_BPD24_atDel','A_FL24_atDel','HC35_atDel','AC35_atDel',
                       'BPD35_atDel','FL35_atDel','A_HC35_atDel','A_AC35_atDel','A_BPD35_atDel','A_FL35_atDel'])


i = 0
initt0_hc, initc_hc,initt0_ac, initc_ac,initt0_bpd,initc_bpd,initt0_fl, initc_fl = 15,1,15,1,15,1,15,1
initA_hc = np.ones(len(X)-1)*300; initA_ac = np.ones(len(X)-1)*300
initA_bpd = np.ones(len(X)-1)*100; initA_fl = np.ones(len(X)-1)*100

for train_init_index, test_init_index in loo.split(X):
    
    X_train_init, X_test_init, y_train_init, y_test_init = X.iloc[train_init_index], X.iloc[test_init_index], y.iloc[train_init_index], y.iloc[test_init_index]                           
    A_hc,t0_hc,c_hc, A_ac,t0_ac,c_ac,A_bpd,t0_bpd,c_bpd,A_fl,t0_fl,c_fl = OptimizeVarAllMeasure(X_train_init,
                                                                        initA_hc,initt0_hc, initc_hc,initA_ac,
                                                                        initt0_ac, initc_ac,initA_bpd,initt0_bpd, 
                                                                        initc_bpd,initA_fl,initt0_fl, initc_fl)
    
    initt0_hc, initc_hc,initt0_ac, initc_ac,initt0_bpd,initc_bpd,initt0_fl, initc_fl = t0_hc,c_hc, t0_ac,c_ac,t0_bpd,c_bpd,t0_fl,c_fl
    initA_hc,initA_ac,initA_bpd,initA_fl = A_hc, A_ac, A_bpd, A_fl

    print('\n Optimization done!!')
    X_train_init = X_train_init.reset_index(drop = True)
    y_train_init = y_train_init.reset_index(drop = True)
    
    X_train_init = predictParam(X_train_init,A_hc,t0_hc,c_hc,A_ac,t0_ac,c_ac,A_bpd,t0_bpd,c_bpd,A_fl,t0_fl,c_fl)
    
    y_test_init = y_test_init.reset_index(drop = True)
    X_test_init = X_test_init.reset_index(drop = True)
    
    index_ga24 = [get_count_GALessThan24(x) for x in X_test_init['GA']]
    index_ga35 = [get_count_GALessThan35(x) for x in X_test_init['GA']]
    gaDel = list(X_test_init['GA_AT_DELIVERY'])
    X_test_init24 = CalA_predBiometries(X_test_init, t0_hc, c_hc, t0_ac, c_ac, t0_bpd,c_bpd, t0_fl, c_fl,
                                      index_ga24, gaDel)
    X_test_init35 = CalA_predBiometries(X_test_init, t0_hc, c_hc, t0_ac, c_ac, t0_bpd,c_bpd, t0_fl, c_fl,
                                      index_ga35, gaDel)
    
    galast = [ga[-1] for ga in X_test_init['GA']]
    
    X_test_init24_last = CalA_predBiometries(X_test_init, t0_hc, c_hc, t0_ac, c_ac, t0_bpd,c_bpd, t0_fl, c_fl,
                                      index_ga24, galast)
    X_test_init35_last = CalA_predBiometries(X_test_init, t0_hc, c_hc, t0_ac, c_ac, t0_bpd,c_bpd, t0_fl, c_fl,
                                      index_ga35, galast)
                       
    temp_df = pd.concat([X_test_init24_last,X_test_init35_last,X_test_init24,X_test_init35],axis=1, join='inner')
    temp_df.columns = ['HC24','AC24','BPD24','FL24','A_HC24','A_AC24','A_BPD24','A_FL24',
                       'HC35','AC35','BPD35','FL35','A_HC35','A_AC35','A_BPD35','A_FL35',
                       'HC24_atDel','AC24_atDel','BPD24_atDel','FL24_atDel','A_HC24_atDel',
                       'A_AC24_atDel','A_BPD24_atDel','A_FL24_atDel','HC35_atDel','AC35_atDel',
                       'BPD35_atDel','FL35_atDel','A_HC35_atDel','A_AC35_atDel','A_BPD35_atDel','A_FL35_atDel']
    
    pred_df = pd.concat([pred_df,temp_df],axis=0, join='inner') 
            
    #alphas = np.arange(0.006, 0.015, 0.001)
    alphas = [0]
    avg_scores = []

    kf = KFold(n_splits=5, shuffle=True, random_state=10)
    
    for a in alphas:
        scores_lasso = []
        
        for train_index, test_index in kf.split(X_train_init):
            
            X_train, X_test, y_train, y_test = X_train_init.iloc[train_index], X_train_init.iloc[test_index], y_train_init.iloc[train_index], y_train_init.iloc[test_index]
            scores_lasso.append(get_score(Lasso(alpha = a,normalize = True), X_train, X_test, y_train, y_test)[0])
        
        avg_scores.append(round(np.average(scores_lasso),6))
    test_output24 = get_score(Lasso(alpha = alphas[avg_scores.index(max(avg_scores))]), X_train_init, X_test_init24,
                           y_train_init,y_test_init)
    
    test_output35 = get_score(Lasso(alpha = alphas[avg_scores.index(max(avg_scores))]), X_train_init, X_test_init35,
                           y_train_init,y_test_init)
    
    df = df.append({'Alpha' : alphas[avg_scores.index(max(avg_scores))], 'Score': test_output24[0],
                    'BWPredicted24' : test_output24[1],'BWPredicted35': test_output35[1], 'BWActual' : y_test_init, 
                    'Parameter_coeff' : test_output24[2],'intercept' : test_output24[3]},ignore_index = True)
    
    i = i+1
    print(i)
    

In [ ]:
df.to_pickle('BWPridiction_LR.pickle')
pred_df.to_pickle('predictedVal.pickle')